In [1]:
from qiskit import QuantumCircuit
from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer


In [31]:
from qiskit.test.mock import (
    FakeBogota,
    FakeCasablanca,
    FakeGuadalupe,
    FakeMontreal,
    FakeManhattan,
) 


In [3]:
# Score generator
# for each MQTbench circuit
    # for each compiler -> tket, staq, qiskit, forest, ...
        # for each backend -> Rigetti, IBM, Google?, AQT?, OQC?, Cirq, ProjectQ
            # generate circuit
            # calculate score
            # safe data: input: qasmfile/gate count per gate, compiler/compiler settings/machine -> output: score

In [4]:
rigetti_m1 = {
    "provider": "rigetti",
    "name": "m1",
    "num_qubits": 80,
    "t1_avg": 33.845,
    "t2_avg": 28.230,
    "avg_gate_time_1q": 60e-3, #source: https://qcs.rigetti.com/qpus -> ASPEN-M-1
    "avg_gate_time_2q": 160e-3 #source: https://qcs.rigetti.com/qpus -> ASPEN-M-1
}
ibm_washington = {
    "provider": "ibm",
    "name": "washington",
    "num_qubits": 127,
    "t1_avg": 103.39,
    "t2_avg": 97.75,
    "avg_gate_time_1q": 206e-3, #estimated, based on the rigetti relation between 1q and 2q and given avg 2q time
    "avg_gate_time_2q": 550.41e-3 #source: https://quantum-computing.ibm.com/services?services=systems&system=ibm_washington
}
ionq = {
    "provider": "ionq",
    "name": "washington",
    "num_qubits": 127,
    "t1_avg": 10000,
    "t2_avg": 0.2,
    "avg_gate_time_1q": 0.00001,
    "avg_gate_time_2q": 0.0002 
}

In [5]:
def count_qubit_gates(qc, provider:str):
    dag = circuit_to_dag(qc)
    count_gates = dag.count_ops_longest_path()
    print(count_gates)
    single_qubit_gates = 0
    two_qubit_gates = 0
    if provider == "ibm":
        # gates: ['id', 'rz', 'sx', 'x', 'cx', 'reset']
        if "id" in count_gates:
            single_qubit_gates += count_gates["id"]
        if "rz" in count_gates:
            single_qubit_gates += count_gates["rz"]
        if "sx" in count_gates:
            single_qubit_gates += count_gates["sx"]
        if "x" in count_gates:
            single_qubit_gates += count_gates["x"]
        if "cx" in count_gates:
            two_qubit_gates += count_gates["cx"]
        
    elif provider == "rigetti":
        # gates: rigetti_native_gates = ["rx", "rz", "cz"]
        if "rx" in count_gates:
            single_qubit_gates += count_gates["rx"]
        if "rz" in count_gates:
            single_qubit_gates += count_gates["rz"]
        if "cz" in count_gates:
            two_qubit_gates += count_gates["cz"]
    return single_qubit_gates, two_qubit_gates
        

In [6]:
def calc_score(filepath, backend):
    if "ibm" in filepath:
        target = "ibm"
    elif "rigetti" in filepath:
        target = "rigetti"
    qc = QuantumCircuit.from_qasm_file(filepath)
    gates_count = count_qubit_gates(qc, target)
    penalty_factor_width = 5000
    penalty_factor_1q = 500
    penalty_factor_2q = 1000

    b_qubits = backend["num_qubits"]
    t_1 = backend["t1_avg"]
    t_2 = backend["t2_avg"]
    avg_gate_time_1q = backend["avg_gate_time_1q"]
    avg_gate_time_2q = backend["avg_gate_time_2q"]
    max_depth_1q = min(t_1, t_2) / avg_gate_time_1q
    max_depth_2q = min(t_1, t_2) / avg_gate_time_2q

    penalty_width = 0
    
    if qc.num_qubits > b_qubits:
        penalty_width = penalty_factor_width
        
    
    score = gates_count[0] / max_depth_1q * penalty_factor_1q + gates_count[1] / max_depth_2q * penalty_factor_2q + penalty_width 
    return score

In [7]:
import os
i = 0
path_to_all_files = "../MQTbench/qasm_output"
res = dict()
for file in os.listdir(path_to_all_files):
    print (file)
    if "ibm" in file:
        score = calc_score(os.path.join(path_to_all_files, file), ibm_washington)
        res[file] = score
    elif "rigetti" in file:
        score= calc_score(os.path.join(path_to_all_files, file), rigetti_m1)
        res[file] = score
    i+=1
    if i>50:
        break

ae_mapped_rigetti-b_opt1_57.qasm
{'rz': 1859, 'rx': 7837, 'cz': 7626, 'barrier': 1, 'measure': 1}
dj_mapped_ibm-b_opt0_51.qasm
{'rz': 8, 'sx': 4, 'x': 3, 'cx': 362, 'barrier': 1, 'measure': 1}
grover-noancilla_nativegates_ibm_opt3_2.qasm
{'rz': 2, 'sx': 1, 'barrier': 1, 'measure': 1}
vqerandom_nativegates_ibm_opt3_3.qasm
{'rz': 6, 'sx': 6, 'cx': 6, 'barrier': 1, 'measure': 1}
vqerandom_mapped_rigetti-b_opt0_4.qasm
{'rz': 423, 'rx': 300, 'cz': 60, 'barrier': 1, 'measure': 1}
grover-noancilla_mapped_ibm-b_opt3_5.qasm
{'rz': 194, 'sx': 112, 'cx': 301, 'x': 5, 'barrier': 1, 'measure': 1}
dj_mapped_rigetti-b_opt0_54.qasm
{'rz': 996, 'rx': 666, 'cz': 289, 'barrier': 1, 'measure': 1}
vqerandom_mapped_rigetti-b_opt2_14.qasm
{'rx': 530, 'rz': 224, 'cz': 360, 'barrier': 1, 'measure': 1}
qftentangled_mapped_rigetti-b_opt1_8.qasm
{'rx': 235, 'rz': 62, 'cz': 228, 'barrier': 1, 'measure': 1}
ae_mapped_ibm-s_opt1_57.qasm
{'rz': 867, 'sx': 227, 'cx': 3080, 'barrier': 1, 'measure': 1}
vqerandom_mapped_

In [8]:
print(res)

{'ae_mapped_rigetti-b_opt1_57.qasm': 53526.036131774716, 'dj_mapped_ibm-b_opt0_51.qasm': 2054.1526342710995, 'grover-noancilla_nativegates_ibm_opt3_2.qasm': 3.161125319693095, 'vqerandom_nativegates_ibm_opt3_3.qasm': 46.42925831202046, 'vqerandom_mapped_rigetti-b_opt0_4.qasm': 1108.395324123273, 'grover-noancilla_mapped_ibm-b_opt3_5.qasm': 2022.5719693094627, 'dj_mapped_rigetti-b_opt0_54.qasm': 3404.179950407368, 'vqerandom_mapped_rigetti-b_opt2_14.qasm': 2841.6578108395324, 'qftentangled_mapped_rigetti-b_opt1_8.qasm': 1607.863974495218, 'ae_mapped_ibm-s_opt1_57.qasm': 18495.598976982095, 'vqerandom_mapped_ibm-s_opt0_14.qasm': 2283.928388746803, 'dj_nativegates_ibm_opt3_59.qasm': 331.8545268542199, 'ae_mapped_ibm-b_opt1_52.qasm': 16326.526035805624, 'vqerandom_mapped_ibm-s_opt2_3.qasm': 113.99877237851662, 'vqerandom_mapped_ibm-b_opt0_11.qasm': 1096.917851662404, 'ae_nativegates_ibm_opt2_53.qasm': 2324.6901278772375, 'vqerandom_mapped_rigetti-s_opt2_11.qasm': 2022.670917463691, 'dj_map

In [9]:
import cirq

def get_cirq_compilation(qasm_qc, target):
    if "ibm" in target:
        pass
    elif "rigetti" in target:
        pass
    elif "ionq" in target:
        pass

In [10]:
from cirq.contrib.qasm_import import circuit_from_qasm
test = open('../MQTbench/qasm_output/vqerandom_indep_5.qasm', 'r').read()
print(test)
circuit = circuit_from_qasm(test)
print(circuit)

// Benchmark was created by qTUMbench on 2022-03-04
// qTUMbench version: 0.1.0
// Qiskit version: {'qiskit-terra': '0.19.1', 'qiskit-aer': '0.10.2', 'qiskit-ignis': '0.7.0', 'qiskit-ibmq-provider': '0.18.3', 'qiskit-aqua': '0.9.5', 'qiskit': '0.34.1', 'qiskit-nature': '0.3.0', 'qiskit-finance': '0.3.0', 'qiskit-optimization': '0.3.0', 'qiskit-machine-learning': '0.3.0'}

OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg meas[5];
ry(0.903070296741003) q[0];
ry(0.827535501355805) q[1];
cx q[0],q[1];
ry(0.0612478491662731) q[2];
cx q[0],q[2];
cx q[1],q[2];
ry(0.177564404963852) q[3];
cx q[0],q[3];
cx q[1],q[3];
cx q[2],q[3];
ry(0.512629131772869) q[4];
cx q[0],q[4];
ry(0.283527993488464) q[0];
cx q[1],q[4];
ry(0.959106709764805) q[1];
cx q[0],q[1];
cx q[2],q[4];
ry(0.341656695006445) q[2];
cx q[0],q[2];
cx q[1],q[2];
cx q[3],q[4];
ry(0.747704427707589) q[3];
cx q[0],q[3];
cx q[1],q[3];
cx q[2],q[3];
ry(0.324987789511013) q[4];
cx q[0],q[4];
ry(0.858556448371372) q[0];
cx q[1],q[4];
ry(

QasmException: Unknown gate "barrier" at line 44

In [13]:
import pennylane as qml
from pennylane import numpy as np

In [14]:
# compilers: tket, qiskit, pennylane, cirq

In [20]:
dev = qml.device("default.qubit", wires=2, shots=5)

@qml.qnode(dev)
@qml.compile()
def qfunc():
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[0, 1])
    qml.sample()

    return qml

compiled_qfunc = qml.compile(
    pipeline=[
        qml.transforms.commute_controlled(direction="left"),
        qml.transforms.merge_rotations(atol=1e-6),
        qml.transforms.cancel_inverses
    ],
    basis_set=["CNOT", "RX", "RY", "RZ"],
    num_passes=2
)(qfunc)

In [44]:
def qfunc(x, y, z):
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[1, 2])
    qml.SX(wires=2)
    qml.CNOT(wires=[2, 1])
    qml.RX(z, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RX(x, wires=0)
    qml.CNOT(wires=[1, 0])
    qml.RZ(-z, wires=2)
    qml.CNOT(wires=[1, 2])
    qml.PauliY(wires=2)
    qml.CNOT(wires=[1, 2])
    return qml.expval(qml.PauliZ(wires=0))

In [45]:
ibm_native_gates = FakeMontreal().configuration().basis_gates
rigetti_native_gates = ["rx", "rz", "cz"]
ibm_native_gates

['id', 'rz', 'sx', 'x', 'cx', 'reset']

In [46]:
dev = qml.device('default.qubit', wires=[0, 1, 2])
qnode = qml.QNode(qfunc, dev)
print(qml.draw(qnode)(0.2, 0.3, 0.4))

 0: ──H───RX(0.4)──────────╭X─────────RX(0.2)──╭X─────────────┤ ⟨Z⟩ 
 1: ──H──╭C────────────╭X──╰C──────────────────╰C──╭C─────╭C──┤     
 2: ─────╰X────────SX──╰C───RZ(-0.4)───────────────╰X──Y──╰X──┤     



In [50]:
compiled_qfunc = qml.compile(basis_set=rigetti_native_gates)(qfunc)
compiled_qnode = qml.QNode(compiled_qfunc, dev)
print(qml.draw(compiled_qnode)(0.2, 0.3, 0.4))

 0: ──RZ(1.57)──RX(1.57)───RZ(1.57)──RX(0.6)──────────────────────────────────────────────────────────────────┤ ⟨Z⟩ 
 1: ──RZ(1.57)──RX(1.57)──╭C─────────RZ(1.57)────────────╭X─────────────╭C────────────────────────────────╭C──┤     
 2: ──────────────────────╰X─────────RZ(1.57)──RY(1.57)──╰C──RZ(-1.97)──╰X──RZ(1.57)──RY(3.14)──RZ(1.57)──╰X──┤     



In [18]:
qnode._circuit.qasm()

AttributeError: 'QNode' object has no attribute '_circuit'

In [9]:
from pytket import Circuit
from pytket.circuit.display import render_circuit_jupyter
c = Circuit(2,2) # define a circuit with 2 qubits and 2 bits
c.H(0)           # add a Hadamard gate to qubit 0
c.Rz(0.25, 0)    # add an Rz gate of angle 0.25*pi to qubit 0
c.CX(1,0)        # add a CX gate with control qubit 1 and target qubit 0
c.measure_all()  # measure qubits 0 and 1, recording the results in bits 0 and 1

render_circuit_jupyter(c)

In [10]:
from pytket.extensions.qiskit import AerBackend
b = AerBackend()                            # connect to the backend
compiled = b.get_compiled_circuit(c)
render_circuit_jupyter(compiled)

In [8]:
from pytket.extensions.pyquil import ForestBackend

from pyquil import get_qc
qvm = get_qc('9q-square-qvm')

b = ForestBackend(qvm)
compiled = b.get_compiled_circuit(c)
render_circuit_jupyter(compiled)

UnknownApiError: 

The server has failed to return a proper response. Please describe the problem
and copy the above message into a GitHub issue at:
    https://github.com/rigetti/pyquil/issues

In [ ]:
from pyquil import get_qc
qvm = get_qc('9q-square-qvm')

In [11]:
from pyquil import get_qc, Program
from pyquil.gates import CNOT, H, MEASURE
 
qvm = get_qc('2q-qvm')
 
p = Program()
p += H(0)
p += CNOT(0, 1)
ro = p.declare('ro', 'BIT', 2)
p += MEASURE(0, ro[0])
p += MEASURE(1, ro[1])
p.wrap_in_numshots_loop(10)

qvm.run(p).readout_data['ro'].tolist()

UnknownApiError: 

The server has failed to return a proper response. Please describe the problem
and copy the above message into a GitHub issue at:
    https://github.com/rigetti/pyquil/issues

In [64]:
with open ("../MQTbench/qasm_output/dj_indep_5.qasm", "r") as myfile:
    data = myfile.read()#.replace('\n', '')
#data = str(data)
print(data, type(data))

// Benchmark was created by qTUMbench on 2022-03-04
// qTUMbench version: 0.1.0
// Qiskit version: {'qiskit-terra': '0.19.1', 'qiskit-aer': '0.10.2', 'qiskit-ignis': '0.7.0', 'qiskit-ibmq-provider': '0.18.3', 'qiskit-aqua': '0.9.5', 'qiskit': '0.34.1', 'qiskit-nature': '0.3.0', 'qiskit-finance': '0.3.0', 'qiskit-optimization': '0.3.0', 'qiskit-machine-learning': '0.3.0'}

OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c[4];
h q[0];
h q[1];
h q[2];
h q[3];
u2(-pi,-pi) q[4];
cx q[0],q[4];
h q[0];
cx q[1],q[4];
h q[1];
cx q[2],q[4];
h q[2];
cx q[3],q[4];
h q[3];
measure q[0] -> c[0];
measure q[1] -> c[1];
measure q[2] -> c[2];
measure q[3] -> c[3];
 <class 'str'>


In [60]:
type((data))

str

In [85]:
my_circuit = qml.from_qasm(data)
my_circuit()

In [83]:
print(qml.draw_mpl(my_circuit))

<function load.<locals>._function at 0x7fecc8607670>
